# NLP Tutorial:Text Classification USing Gensim Word Embeddings

In [1]:
# !pip install spacy
# pip install spacy-download

In [7]:
!pip3 install python spacy download en_core_web_lg

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement python (from versions: none)
ERROR: No matching distribution found for python


In [9]:
import gensim.downloader as api
import pandas as pd
import spacy
import gensim
from spacy_download import load_spacy

In [10]:
spacy.__version__

'3.4.4'

In [11]:
gensim.__version__

'4.2.0'

In [12]:
print(list(gensim.downloader.info()['models'].keys()))

ValueError: unable to read local cache '/home/sangramdh/gensim-data/information.json' during fallback, connect to the Internet and retry

In [ ]:
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)

In [ ]:
#read the dataset with name "cyber.csv" and store it in a variable df
cyber = pd.read_csv("/home/sangramdh/jupyter/NLP/text_classification/data/cybersecurity_tweets.csv", usecols=['text','label'])

#print the shape of dataframe
print(cyber.shape)

#print top 5 rows
cyber.head(555)

In [ ]:
#read the dataset with name "non_cyber.csv" and store it in a variable df
non_cyber = pd.read_csv("/home/sangramdh/jupyter/NLP/text_classification/data/not_cybersecurity_tweets.csv", usecols=['text', 'label'])

#print the shape of dataframe
print(non_cyber.shape)

#print top 5 rows
non_cyber.head()

### Check uniques values in "label" column

In [ ]:
cyber.label.unique() # for cyber dataset

In [ ]:
cyber[-10:]

In [ ]:
len(cyber.label.unique()) # check count how many uniques are there

In [ ]:
non_cyber.label.unique() # check for non_cyber dataset

In [ ]:
len(non_cyber.label.unique())

#### I'm getting here at label column in both the dataset, some unique which is not good.
we need to clean that so that we can move forword

In [ ]:
cyber['label'] = cyber.label.replace(cyber.label.unique(), 1) # replacing the value for cyber dataset

In [ ]:
cyber_labeled = cyber

In [ ]:
cyber_labeled.head()

In [ ]:
len(cyber_labeled.label.unique())

In [ ]:
non_cyber['label'] = non_cyber.label.replace(non_cyber.label.unique(), 0)

In [ ]:
non_cyber_labeled = non_cyber

In [ ]:
non_cyber_labeled.head()

In [ ]:
len(non_cyber_labeled.label.unique())

In [ ]:
#check the distribution of labels 
print(non_cyber_labeled['label'].value_counts())

### concanet the data frame => cyber_labeled + non_cyber_labeled = data

In [ ]:

# concatenating df1 and df2 along rows
data = pd.concat([cyber_labeled, non_cyber_labeled], axis=0)
print(data.shape)
data.head()

In [ ]:
#check the distribution of labels 
print(data['label'].value_counts())

In [ ]:
len(data.label.unique())

#### ```Now we have propely labeled data we can move forward for making vectors```

In [ ]:
r1 = np.mean([wv_good, wv_great],axis=0) # I was checking real mean is similar to "get_mean_vactor" or not

In [ ]:
wv_good[:5]

In [ ]:
wv_great[:5]

In [ ]:
r1[:5]

In [ ]:
r2 = wv.get_mean_vector(["good", "great"],pre_normalize=False)

In [ ]:
r2[:5]

### Preprocess and make them a vector

In [ ]:
def preprocess(text):
    text = re.sub(r'[^\w\s\']', ' ', text)
    text = re.sub(r'[- \n]+', ' ', text)
    text = re.sub(r"http\S+", "", text)
    return text.strip().lower()

In [7]:
python3 -m spacy download en_core_web_sm

SyntaxError: invalid syntax (2180754472.py, line 1)

In [ ]:
import spacy.cli
spacy.cli.download("en_core_web_lg")

In [1]:
import spacy
nlp = spacy.load("en_core_web_lg")

2022-12-28 12:32:56.442787: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-28 12:32:57.185630: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-28 12:32:57.185719: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-28 12:32:57.185731: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

OSError: [E050] Can't find model 'en_core_web_lg'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
nlp = load_spacy("en_core_web_lg", exclude=["parser", "tagger"])

In [ ]:
def preprocess_and_vectorize(text):
    # remove stop words and lemmatize the text
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
        
    return wv.get_mean_vector(filtered_tokens)

In [ ]:
v = preprocess_and_vectorize("Don't worry if you don't understand")
v.shape

In [ ]:
#this query takes few minutes, so go get some walk :)

df['vector'] = df['Text'].apply(lambda text: preprocess_and_vectorize(text))

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split


#Do the 'train-test' splitting with test size of 20% with random state of 2022 and stratify sampling too
X_train, X_test, y_train, y_test = train_test_split(
    df.vector.values, 
    df.label_num, 
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2022,
    stratify=df.label_num
)

In [ ]:
print("Shape of X_train before reshaping: ", X_train.shape)
print("Shape of X_test before reshaping: ", X_test.shape)


X_train_2d = np.stack(X_train)
X_test_2d =  np.stack(X_test)

print("Shape of X_train after reshaping: ", X_train_2d.shape)
print("Shape of X_test after reshaping: ", X_test_2d.shape)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

#1. creating a GradientBoosting model object
clf = GradientBoostingClassifier()

#2. fit with all_train_embeddings and y_train
clf.fit(X_train_2d, y_train)


#3. get the predictions for all_test_embeddings and store it in y_pred
y_pred = clf.predict(X_test_2d)


#4. print the classfication report
print(classification_report(y_test, y_pred))

In [ ]:
test_news = [
    "Michigan governor denies misleading U.S. House on Flint water (Reuters) - Michigan Governor Rick Snyder denied Thursday that he had misled a U.S. House of Representatives committee last year over testimony on Flintâ€™s water crisis after lawmakers asked if his testimony had been contradicted by a witness in a court hearing. The House Oversight and Government Reform Committee wrote Snyder earlier Thursday asking him about published reports that one of his aides, Harvey Hollins, testified in a court hearing last week in Michigan that he had notified Snyder of an outbreak of Legionnairesâ€™ disease linked to the Flint water crisis in December 2015, rather than 2016 as Snyder had testified. â€œMy testimony was truthful and I stand by it,â€ Snyder told the committee in a letter, adding that his office has provided tens of thousands of pages of records to the committee and would continue to cooperate fully.  Last week, prosecutors in Michigan said Dr. Eden Wells, the stateâ€™s chief medical executive who already faced lesser charges, would become the sixth current or former official to face involuntary manslaughter charges in connection with the crisis. The charges stem from more than 80 cases of Legionnairesâ€™ disease and at least 12 deaths that were believed to be linked to the water in Flint after the city switched its source from Lake Huron to the Flint River in April 2014. Wells was among six current and former Michigan and Flint officials charged in June. The other five, including Michigan Health and Human Services Director Nick Lyon, were charged at the time with involuntary manslaughter",
    " WATCH: Fox News Host Loses Her Sh*t, Says Investigating Russia For Hacking Our Election Is Unpatriotic This woman is insane.In an incredibly disrespectful rant against President Obama and anyone else who supports investigating Russian interference in our election, Fox News host Jeanine Pirro said that anybody who is against Donald Trump is anti-American. Look, it s time to take sides,  she began.",
    " Sarah Palin Celebrates After White Man Who Pulled Gun On Black Protesters Goes Unpunished (VIDEO) Sarah Palin, one of the nigh-innumerable  deplorables  in Donald Trump s  basket,  almost outdid herself in terms of horribleness on Friday."
]

test_news_vectors = [preprocess_and_vectorize(n) for n in test_news]
clf.predict(test_news_vectors)

In [ ]:
#finally print the confusion matrix for the best model (GradientBoostingClassifier)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm


from matplotlib import pyplot as plt
import seaborn as sn
plt.figure(figsize = (10,7))
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Prediction')
plt.ylabel('Truth')